In [21]:
%reload_ext autoreload
# %load_ext autoreload
%autoreload 2


# Sourcing parameter information

At this point of time, the JS side did not implement a single source our truth (SSOT) architecture, 
we won't be able to source our truth from the same place. 
So we won't be doing real SSOT here, but will try to do so at least on the python side, 
by sourcing from various key places of the JS side, and create our own master SSOT.

The purpose of the `cosmograph._resources` module (whose main object is `ConfigsDacc`), 
demoed in the following section is to create, and maintain (therefore, source, diagnose, edit, etc.) 
a master SSOT json file for the python interface. From this file, we'll draw what the names of our parameters/arguments
are, as well as their type annotations, defaults, and descriptions. 

Our SSOT file is 

In [157]:
from cosmograph.util import cosmograph_base_signature, cosmograph_base_docs, PARAMS_SSOT_PATH, _params_ssot

# The ssot file is stored locally with the package. It's path is given by PARAMS_SSOT_PATH
print(f"{PARAMS_SSOT_PATH=}")

PARAMS_SSOT_PATH=PosixPath('/Users/thorwhalen/Dropbox/py/proj/c/cosmograph/cosmograph/data/params_ssot.json')


In [158]:
# To extract the ssot information, we use the _params_ssot function
params = _params_ssot()
params[:2]  # see the first two entries

[{'name': 'disable_simulation',
  'default': False,
  'annotation': 'bool',
  'description': 'Prevents the simulation from running, merely rendering the graph.'},
 {'name': 'simulation_decay',
  'default': 1000,
  'annotation': 'float',
  'description': 'Defines how quickly the simulation cools down.'}]

In [159]:
# We use the params to extract the descriptions of the various parameters we use in cosmograph, 
# and therefore be able to generate the part of the documentation that lists and describtes them
print(cosmograph_base_docs()[:200], '...')

Parameters
----------
disable_simulation : bool, default=False
    Prevents the simulation from running, merely rendering the graph.
simulation_decay : float, default=1000
    Defines how quickly the  ...


In [160]:
# We also use params to make a propert signature for our functions (instead of relying on uninformative *args and **kwargs)
print(cosmograph_base_signature())

(*, disable_simulation: bool = False, simulation_decay: float = 1000, simulation_gravity: float = 0, simulation_center: float = 0, simulation_repulsion: float = 0.1, simulation_repulsion_theta: float = 1.7, simulation_repulsion_quadtree_levels: float = 12, simulation_link_spring: float = 1, simulation_link_distance: float = 2, simulation_link_dist_random_variation_range: list[typing.Any] = [1, 1.2], simulation_repulsion_from_mouse: float = 2, simulation_friction: float = 0.85, simulation_cluster: float = None, background_color: Union[str, list[float]] = '#222222', space_size: int = 4096, point_color: Union[str, list[float]] = '#b3b3b3', point_greyout_opacity: float = 0.1, point_size: float = 4, point_size_scale: float = 1, hovered_point_cursor: str = None, render_hovered_point_ring: bool = 0.7, hovered_point_ring_color: Union[str, list[float]] = 'white', focused_point_ring_color: Union[str, list[float]] = 0.95, focused_point_index: int = None, render_links: bool = True, link_color: Uni

## ConfigsDacc

The `ConfigsDacc` manages access (and creation, and diagnosis) of the configs that 
constitute the interface of cosmograph visualization functionalities. 

Many of it's datas have two sides to them: 
* The side that acquires the data from some fixed sources (that may get updated sometimes)
* The side that saves prepared, ready to use, data. 

Here are the various kinds of datas (and their sources):

* **Names**: The names from the [traitlets definition in the cosmograph_widget Cosmograph class](https://github.com/cosmograph-org/cosmograph_widget/blob/dev/src/cosmograph_widget/widget.py), since these are the only names that are defined in the widget object that bridges with JS.
* **Annotations**: These will also use the [traitlets definition in the cosmograph_widget Cosmograph class](https://github.com/cosmograph-org/cosmograph_widget/blob/dev/src/cosmograph_widget/widget.py), since these are the actual ones defining the python types. These have been defined manually, but in a later development, we should define these via a type mapping logic based on the types taken from the TS files (see below)
* **Defaults**: From the [cosmos/.../variables.ts](https://github.com/cosmograph-org/cosmos/blob/main/src/variables.ts) file of the main branch. This file uses a standard, `export const NAME = DEFAULT_VALUE` format, so can robustly be parsed. 
* **Kinds**: All kinds will be (as they are in the base `Cosmograph` class) keyword-only, though one or two may be converted to keyword-or-position later.
* **Descriptions**: Could be from the TS files above, and/or from md docs. Permanent source to be determined, but currently was pointed to [this commit](https://github.com/cosmograph-org/cosmograph/blob/8c868898f256a207e12085a17566943bea49c28a/packages/website/pages/docs/v2/widget/configuration.mdx)

Extras:
* **Types**: From the five TS files;
   * from default branch: [cosmos/config.ts](https://github.com/cosmograph-org/cosmos/blob/next/src/config.ts)
   * from cosmograph/config dev branch; [config.ts](https://github.com/cosmograph-org/cosmograph/blob/dev/packages/cosmograph/src/cosmograph/config/config.ts), [data.ts](https://github.com/cosmograph-org/cosmograph/blob/dev/packages/cosmograph/src/cosmograph/config/data.ts), [labels.ts](https://github.com/cosmograph-org/cosmograph/blob/dev/packages/cosmograph/src/cosmograph/config/labels.ts), [simulation.ts](https://github.com/cosmograph-org/cosmograph/blob/dev/packages/cosmograph/src/cosmograph/config/simulation.ts)

It should be noted that these sources all have opinions on more than one interface feature. 
They all have names, and often defaults and types. 
We will only use these, at this point, for misalignment diagnoses, until we, one day, get a proper SoT setup.



In [19]:
from cosmograph._resources import ConfigsDacc
import pandas as pd

c = ConfigsDacc()

print(f"{len(c.source_strings)=}")
print(f"{list(c.source_strings)=}")
print(f"Example of value: '{next(iter(c.source_strings.values()))[:40]}...'")


len(c.source_strings)=7
list(c.source_strings)=['cosmos/variables.ts', 'cosmos/config.ts', 'cosmograph/config.ts', 'cosmograph/data.ts', 'cosmograph/labels.ts', 'cosmograph/simulation.ts', 'cosmograph/configuration.mdx']
Example of value: 'export const defaultNodeColor = '#b3b3b3...'


The `info_dfs()` yields `(name, dataframe)` pairs for the different sources of information we'll be sourcing from.

In [20]:
t = dict(c.info_dfs())
list(t)

['traitlets', 'defaults', 'descriptions', 'types']

The `matched_info_df` dataframe joins these (keeping only those names that are in the traitlets, which defines what is actually accessible to the python interface)

In [21]:
c.matched_info_df

,traitlet_annotation,defaults_default,ts_types_default,md_descriptions_default,md_descriptions_description,ts_types_description,ts_types_type,traitlet_default,md_descriptions_type,md_descriptions_group,md_descriptions_optional,ts_types_name,ts_types_optional,ts_types_py_name,ts_types_origin_name
name,,,,,,,,,,,,,,,
disable_simulation,<class 'bool'>,False,NaN,None,"Prevents the simulation from running, merely r...","Do not run the simulation, just render the gra...",boolean,None,boolean,Simulation Settings,True,disableSimulation,True,disable_simulation,GraphConfigInterface
simulation_decay,<class 'float'>,1000,NaN,1000,Defines how quickly the simulation cools down.,Decay coefficient for simulation cool down rat...,number,None,number,Simulation Settings,True,simulationDecay,True,simulation_decay,GraphConfigInterface
simulation_gravity,<class 'float'>,0,NaN,0,Coefficient for gravity force.,Gravity force coefficient. Default `0.25`.,number,None,number,Simulation Settings,True,simulationGravity,True,simulation_gravity,GraphConfigInterface
simulation_center,<class 'float'>,0,NaN,0,Centers the mass force coefficient.,Centering to center mass force coefficient. De...,number,None,number,Simulation Settings,True,simulationCenter,True,simulation_center,GraphConfigInterface
simulation_repulsion,<class 'float'>,0.1,NaN,0.1,Configures point repulsion between points.,Repulsion force coefficient. Default `1.0`.,number,None,number,Simulation Settings,True,simulationRepulsion,True,simulation_repulsion,GraphConfigInterface
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
clicked_point_id,<class 'str'>,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selected_point_indices,list[int],NaN,NaN,NaN,NaN,NaN,NaN,traitlets.Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selected_point_ids,list[str],NaN,NaN,NaN,NaN,NaN,NaN,traitlets.Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
c.matched_info_df.loc['render_hovered_point_ring']

traitlet_annotation                                               <class 'bool'>
defaults_default                                                             0.7
ts_types_default                                                             NaN
md_descriptions_default                                                    False
md_descriptions_description                  Enables ring around hovered points.
ts_types_description           Rings rendering around a point on hover toggle...
ts_types_type                                                            boolean
traitlet_default                                                            None
md_descriptions_type                                                     boolean
md_descriptions_group                                           Graph Appearance
md_descriptions_optional                                                    True
ts_types_name                                             renderHoveredPointRing
ts_types_optional           

## Example usage of these tables (exporting them for docs, making signatures, etc.)

In [ ]:
# copy markdown version of matched_info_df to clipboard
__import__('pyperclip').copy(c.matched_info_df_prepared_for_export().to_markdown())


In [3]:
# copy csv version of matched_info_df to clipboard
__import__('pyperclip').copy(c.matched_info_df_prepared_for_export().to_csv(sep='\t'))

In [8]:
# The signature of the base class, made from the matched_info_df information
sig = c.cosmograph_base_signature()
sig

<Sig (*, disable_simulation: bool = False, simulation_decay: float = 1000, simulation_gravity: float = 0, simulation_center: float = 0, simulation_repulsion: float = 0.1, simulation_repulsion_theta: float = 1.7, simulation_repulsion_quadtree_levels: float = 12, simulation_link_spring: float = 1, simulation_link_distance: float = 2, simulation_link_dist_random_variation_range: list[typing.Any] = [1, 1.2], simulation_repulsion_from_mouse: float = 2, simulation_friction: float = 0.85, background_color: Union[str, list[float]] = '#222222', space_size: int = 4096, default_point_color: Union[str, list[float]] = nan, point_greyout_opacity: float = 0.1, default_point_size: float = nan, point_size_scale: float = 1, hovered_point_cursor: str = nan, render_hovered_point_ring: bool = 0.7, hovered_point_ring_color: Union[str, list[float]] = 'white', focused_point_ring_color: Union[str, list[float]] = 0.95, focused_point_index: int = nan, render_links: bool = True, default_link_color: Union[str, lis

In [ ]:
# Make the docs (params info part) for the base class
len(c.cosmograph_base_docs())

7588

## Diagnoses

In [23]:
c.print_diagnosis()


Duplicates of types group:

                                                           0                      1
disableSimulation                       GraphConfigInterface            BasicConfig
simulationDecay                         GraphConfigInterface       SimulationConfig
simulationGravity                       GraphConfigInterface       SimulationConfig
simulationCenter                        GraphConfigInterface       SimulationConfig
simulationRepulsion                     GraphConfigInterface       SimulationConfig
simulationRepulsionTheta                GraphConfigInterface       SimulationConfig
simulationRepulsionQuadtreeLevels       GraphConfigInterface       SimulationConfig
simulationLinkSpring                    GraphConfigInterface       SimulationConfig
simulationLinkDistance                  GraphConfigInterface       SimulationConfig
simulationLinkDistRandomVariationRange  GraphConfigInterface       SimulationConfig
simulationRepulsionFromMouse            GraphCo

# Updating the configs

But how did we get these resources and configs data?
How do we update them?

We'll just take care of configs here. 
`ConfigDacc` is still your go-to tool. 
It's built to do everything it does in a lazy-eval fashion. 

When you do `current = ConfigsDacc()` what you get is an instance that points to the default place where prepared config data is saved. 
One thing you can do is just delete those files, and as you try to access configuration data,
the `ConfigsDacc` instance will recreate them (fetch the original sources online, process them, and finally save the prepared data). 

But we prefer the following method: Staging the updated data, then comparing to the current configuration data, and finally replacing it if all looks okay.

So the first thing we'll do is select a temporary folder to put our staging data. 
You can do this however you want: Here we'll use a temporary folder for this.

In [24]:
import os
from dol import temp_dir


if 'staging_dir' not in locals():
    # if the staging_dir is not already defined, create a new one
    # Note: If you need a new dir, delete or empty the old one first
    staging_dir = temp_dir('cosmograph_config_staging_dir')

_staging_dir_contents = list(os.listdir(staging_dir))
if _staging_dir_contents:
    print(f"Warning: The staging dir is not empty: {_staging_dir_contents}")

staging_dir

'/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/cosmograph_config_staging_dir'

Now you can make two instances of `ConfigsDacc`: One pointing to the current data, 
and the other to `staging_dir`, where you'll renew the configs.

In [25]:
from cosmograph._resources import ConfigsDacc

current = ConfigsDacc()
stage = ConfigsDacc(staging_dir)

Now what we'll do is compare both `current` and `stage`, and if and when we feel like the `stage` is good, we can copy the files over.

See that part below in the "Replacing current with stage".

In [ ]:
current.matched_info_df

,traitlet_annotation,defaults_default,ts_types_default,md_descriptions_default,md_descriptions_description,ts_types_description,ts_types_type,traitlet_default,md_descriptions_type,md_descriptions_group,md_descriptions_optional,ts_types_name,ts_types_optional,ts_types_py_name,ts_types_origin_name
name,,,,,,,,,,,,,,,
disable_simulation,<class 'bool'>,False,NaN,None,"Prevents the simulation from running, merely r...","Do not run the simulation, just render the gra...",boolean,None,boolean,Simulation Settings,True,disableSimulation,True,disable_simulation,GraphConfigInterface
simulation_decay,<class 'float'>,1000,NaN,1000,Defines how quickly the simulation cools down.,Decay coefficient for simulation cool down rat...,number,None,number,Simulation Settings,True,simulationDecay,True,simulation_decay,GraphConfigInterface
simulation_gravity,<class 'float'>,0,NaN,0,Coefficient for gravity force.,Gravity force coefficient. Default `0.25`.,number,None,number,Simulation Settings,True,simulationGravity,True,simulation_gravity,GraphConfigInterface
simulation_center,<class 'float'>,0,NaN,0,Centers the mass force coefficient.,Centering to center mass force coefficient. De...,number,None,number,Simulation Settings,True,simulationCenter,True,simulation_center,GraphConfigInterface
simulation_repulsion,<class 'float'>,0.1,NaN,0.1,Configures point repulsion between points.,Repulsion force coefficient. Default `1.0`.,number,None,number,Simulation Settings,True,simulationRepulsion,True,simulation_repulsion,GraphConfigInterface
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
clicked_point_id,<class 'str'>,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selected_point_indices,list[int],NaN,NaN,NaN,NaN,NaN,NaN,traitlets.Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selected_point_ids,list[str],NaN,NaN,NaN,NaN,NaN,NaN,traitlets.Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
stage.matched_info_df # takes ~2 minutes to run the first time

[autoreload of cosmograph._resources failed: Traceback (most recent call last):
  File "/Users/thorwhalen/Dropbox/py/proj/i/dol/dol/filesys.py", line 276, in wrapped_method
    return func(self, k, *args, **kwargs)
  File "/Users/thorwhalen/Dropbox/py/proj/i/dol/dol/filesys.py", line 480, in __getitem__
    with open(k, **self._read_open_kwargs) as fp:
FileNotFoundError: [Errno 2] No such file or directory: '/Users/thorwhalen/Dropbox/py/proj/c/py_cosmograph/cosmograph/data/config_info.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/thorwhalen/.pyenv/versions/3.10.13/envs/p10/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/thorwhalen/.pyenv/versions/3.10.13/envs/p10/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/thorwhalen/.

,traitlet_annotation,defaults_default,ts_types_default,md_descriptions_default,md_descriptions_description,ts_types_description,ts_types_type,traitlet_default,md_descriptions_type,md_descriptions_group,md_descriptions_optional,ts_types_name,ts_types_optional,ts_types_py_name,ts_types_origin_name,ts_types_properties
name,,,,,,,,,,,,,,,,
disable_simulation,<class 'bool'>,False,NaN,None,"Prevents the simulation from running, merely r...","Do not run the simulation, just render the gra...",boolean,None,boolean,Simulation Settings,True,disableSimulation,True,disable_simulation,GraphConfigInterface,NaN
simulation_decay,<class 'float'>,5000,NaN,1000,Defines how quickly the simulation cools down.,Decay coefficient. Default value: `5000`,number,None,number,Simulation Settings,True,simulationDecay,True,simulation_decay,GraphConfigInterface,NaN
simulation_gravity,<class 'float'>,0.25,NaN,0,Coefficient for gravity force.,Gravity force coefficient. Default value: `0.25`,number,None,number,Simulation Settings,True,simulationGravity,True,simulation_gravity,GraphConfigInterface,NaN
simulation_center,<class 'float'>,0,NaN,0,Centers the mass force coefficient.,Centering to center mass force coefficient. De...,number,None,number,Simulation Settings,True,simulationCenter,True,simulation_center,GraphConfigInterface,NaN
simulation_repulsion,<class 'float'>,1,NaN,0.1,Configures point repulsion between points.,Repulsion force coefficient. Default value: `1.0`,number,None,number,Simulation Settings,True,simulationRepulsion,True,simulation_repulsion,GraphConfigInterface,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
clicked_point_id,<class 'str'>,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selected_point_indices,list[int],NaN,NaN,NaN,NaN,NaN,NaN,traitlets.Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selected_point_ids,list[str],NaN,NaN,NaN,NaN,NaN,NaN,traitlets.Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# stage.matched_info_df.loc['render_hovered_point_ring']

In [62]:
from cosmograph._resources import source_url_groups

def parsed_defaults(self):
    from jy import variable_declarations_pairs

    for k in source_url_groups['defaults']:
        yield k, dict(variable_declarations_pairs(self.source_strings[k]))

t = dict(parsed_defaults(stage))

In [63]:
t

{'cosmos/variables.ts': {'defaultPointColor': '#b3b3b3',
  'defaultGreyoutPointOpacity': 0.1,
  'defaultPointSize': 4,
  'defaultLinkColor': '#666666',
  'defaultGreyoutLinkOpacity': 0.1,
  'defaultLinkWidth': 1,
  'defaultBackgroundColor': '#222222',
  'defaultConfigValues': {'disableSimulation': False,
   'spaceSize': 8192,
   'pointSizeScale': 1,
   'linkWidthScale': 1,
   'arrowSizeScale': 1,
   'renderLinks': True,
   'curvedLinks': False,
   'curvedLinkSegments': 19,
   'curvedLinkWeight': 0.8,
   'curvedLinkControlPointDistance': 0.5,
   'arrowLinks': False,
   'linkVisibilityDistanceRange': [50, 150],
   'linkVisibilityMinTransparency': 0.25,
   'hoveredPointCursor': 'auto',
   'renderHoveredPointRing': False,
   'hoveredPointRingColor': 'white',
   'focusedPointRingColor': 'white',
   'focusedPointIndex': None,
   'useQuadtree': False,
   'simulation': {'decay': 5000,
    'gravity': 0.25,
    'center': 0,
    'repulsion': 1,
    'repulsionTheta': 1.15,
    'repulsionQuadtreeLe

'/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/cosmograph_config_staging_dir/config_prep'

In [ ]:
from pyperclip import copy
from cosmograph import Cosmograph
copy(stage.matched_info_df.fillna('').to_markdown())

### What parameters are not documented?

In [29]:
md_descriptions = stage.matched_info_df.md_descriptions_description
missing_md_descriptions = md_descriptions[md_descriptions.isna()].fillna('').drop(['_ipc_points', '_ipc_links'])
print(f"{len(missing_md_descriptions)} missing descriptions")
missing_md_descriptions


34 missing descriptions


name
simulation_cluster            
link_arrows                   
point_id_by                   
point_index_by                
point_color_by                
point_size_by                 
point_size_range              
point_label_by                
point_label_weight_by         
point_x_by                    
point_y_by                    
point_cluster_by              
point_cluster_strength_by     
link_source_by                
link_source_index_by          
link_target_by                
link_target_index_by          
link_color_by                 
link_width_by                 
link_arrow_by                 
link_strength_by              
link_strength_range           
show_labels                   
label_margin                  
disable_point_size_legend     
disable_link_width_legend     
disable_point_color_legend    
disable_link_color_legend     
clicked_point_index           
clicked_point_id              
selected_point_indices        
selected_point_ids            
cha

In [30]:
ts_descriptions = stage.matched_info_df.ts_types_description
missing_ts_descriptions = ts_descriptions[ts_descriptions.isna()].fillna('').drop(['_ipc_points', '_ipc_links'])
print(f"{len(missing_ts_descriptions)} missing descriptions")
missing_ts_descriptions

11 missing descriptions


name
show_FPS_monitor              
disable_point_size_legend     
disable_link_width_legend     
disable_point_color_legend    
disable_link_color_legend     
clicked_point_index           
clicked_point_id              
selected_point_indices        
selected_point_ids            
changePoints                  
changeLinks                   
Name: ts_types_description, dtype: object

In [37]:
set(missing_ts_descriptions.index) & set(missing_md_descriptions.index)

{'changeLinks',
 'changePoints',
 'clicked_point_id',
 'clicked_point_index',
 'disable_link_color_legend',
 'disable_link_width_legend',
 'disable_point_color_legend',
 'disable_point_size_legend',
 'selected_point_ids',
 'selected_point_indices'}

In [35]:
set(missing_ts_descriptions.index) - set(missing_md_descriptions.index)

{'show_FPS_monitor'}

In [54]:
# get duplicates of stage.matched_info_df.ts_types_name
duplicates = stage.matched_info_df.ts_types_name[stage.matched_info_df.ts_types_name.duplicated(keep=False)]
stage.matched_info_df.ts_types_name

name
disable_simulation          disableSimulation
simulation_decay              simulationDecay
simulation_gravity          simulationGravity
simulation_center            simulationCenter
simulation_repulsion      simulationRepulsion
                                 ...         
clicked_point_id                          NaN
selected_point_indices                    NaN
selected_point_ids                        NaN
changePoints                              NaN
changeLinks                               NaN
Name: ts_types_name, Length: 97, dtype: object

In [57]:
# get a "py_for_ts_name" to map ts_types_name to the parameter names we're using in python
t = stage.matched_info_df.dropna(subset=['ts_types_name'])
py_for_ts_name = dict(zip(t.ts_types_name, t.index))
assert len(py_for_ts_name) == len(t), "some names weren't unique!"
len(py_for_ts_name)

84

In [65]:
ts_names_that_have_descriptions_we_can_use_for_md = sorted(set(missing_md_descriptions.index) - set(missing_ts_descriptions.index))
alternate_descriptions = ts_descriptions.loc[ts_names_that_have_descriptions_we_can_use_for_md]
# go through the alternate_descriptions values (strings) and
# replace any ts_names we find (any of the keys of py_for_ts_name) with the corresponding python name
from lkj import regex_based_substitution
find_replace = regex_based_substitution(py_for_ts_name)
alternate_descriptions = alternate_descriptions.apply(find_replace)

print(f"{len(alternate_descriptions)} recoverable descriptions")
print(alternate_descriptions.to_markdown())

24 recoverable descriptions
| name                      | ts_types_description                                                                                                     |
|:--------------------------|:-------------------------------------------------------------------------------------------------------------------------|
| label_margin              | Specifies the margin between the label and the point.                                                                    |
| link_arrow_by             | Column name that determines whether a link should have an arrow.                                                         |
| link_arrows               | Control displaying link arrows. Default `false`.                                                                         |
| link_color_by             | Column name for the link color. Links will be colored based on this column.                                              |
| link_source_by            | Column name for the sour

## Diagnose the differences between current and stage

If you're here, this means you're an advanced cosmograph developer. 
Congradulations!

It's aďsumed therefore that you have an idea of what the changes of the cosmograph interface have been
(i.e. parameter names, descriptions, defaults, types). 

You should use that privileged knowledge to verify if everything that should be still there is still there in 
stage, what shouldn't be there is not, and that the new stuff is present...

In [5]:
stage_names = set(stage.matched_info_df.index)
len(stage_names)

97

### Compare names

Note that you shouldn't have any differences between current and stage property names 
(i.e. dataframe indices). 

This is because they both source from the same place, dynamically (not through a config file): 
From the `cosmograph_widget.Cosmograph` traitlets. 

In [61]:
assert set(current.matched_info_df.index) - set(stage.matched_info_df.index) == set()
assert set(stage.matched_info_df.index) - set(current.matched_info_df.index) == set()

But fret not! If you want to actually compare some other traitlet names 
(other than the ones of the `cosmograph_widget.Cosmograph` of the environment you're running this on)
you can simply specify an explicit `Cosmograph` class you want to use. 

One way you can do this is by specifying the **raw** (important!!) url of the module it's defined. 
See the example below:

In [85]:
raw_url_of_previous_widget_module = (
    'https://raw.githubusercontent.com/cosmograph-org/cosmograph_widget/e807bb829c33ce6f87ba16b63616a9b24d4c7d17/src/cosmograph_widget/__init__.py'
)
previous = ConfigsDacc(cosmograph_widget_source=raw_url_of_previous_widget_module)


In [102]:
in_previous_not_in_stage = set(previous.matched_info_df.index) - set(stage.matched_info_df.index)
in_stage_not_in_previous = set(stage.matched_info_df.index) - set(previous.matched_info_df.index)

from lkj import wrapped_print

print("In previous not in stage:")
wrapped_print(in_previous_not_in_stage)
print("\nIn stage not in previous:")
wrapped_print(in_stage_not_in_previous)


In previous not in stage:
default_point_color, point_index, link_target, point_id, default_link_arrows,
default_link_width, point_label_weight, point_label, link_strength,
default_point_size, default_link_color, link_source, label_padding, point_y,
link_source_index, point_x, link_arrow, link_target_index

In stage not in previous:
link_color_by, link_arrow_by, point_cluster_strength_by, link_target_index_by,
link_target_by, point_label_by, point_color_by, point_label_weight_by,
point_size_by, point_id_by, point_size_range, point_index_by, point_x_by,
label_margin, show_labels, link_source_index_by, simulation_cluster,
link_strength_range, link_source_by, point_cluster_by, link_width_by,
link_strength_by, point_y_by, link_arrows


### Signature differences beyond the names

Now, taking the current and stage names (which are the same), what you probably want to do is 
see what the differences in non-traitlet columns (columns prefixed by `ts_` and `md_`). 

In [5]:
from cosmograph._resources import sig_to_df, signature_diffs
from tabled import dataframe_diffs

sig_diffs = signature_diffs(current.cosmograph_base_signature(), stage.cosmograph_base_signature())
sig_diffs

{}

In [6]:
current.cosmograph_base_docs() == stage.cosmograph_base_docs()

True

### ... and more

In [158]:
from tabled import dataframe_diffs

def rename_columns(df):
    return df.rename(columns={'left': 'current', 'right': 'stage'})

t = dataframe_diffs(current.matched_info_df, stage.matched_info_df)
list(t['columns_value_diff'])

['ts_types_default',
 'ts_types_description',
 'ts_types_type',
 'md_descriptions_type',
 'ts_types_name',
 'ts_types_optional',
 'ts_types_py_name',
 'ts_types_origin_name']

In [162]:
rename_columns(t['columns_value_diff']['ts_types_default'])

,current,stage
name,,
disable_simulation,False,NaN
simulation_decay,5000,NaN
simulation_gravity,0.25,NaN
simulation_center,0,NaN
simulation_repulsion,1.0,NaN
simulation_repulsion_theta,1.15,NaN
simulation_repulsion_quadtree_levels,12,NaN
simulation_link_spring,1,NaN
simulation_link_distance,10,NaN


In [138]:
def differences_for_name(name, *, current=current, stage=stage):
    t = pd.concat(
        [
            current.matched_info_df.loc[name].rename('current'),
            stage.matched_info_df.loc[name].rename('stage'),
        ],
        axis=1,
    )
    t = t.dropna(how='all')  # because no NaNs resolve as equal
    t['is_same'] = t['current'] == t['stage']
    t = t[t['is_same'] == False]
    del t['is_same']
    return t


In [142]:
differences_for_name('link_color')

,current,stage
ts_types_description,NaN,Default color for links. Either hex or RGBA. D...
ts_types_type,NaN,"string | [number, number, number, number]"
ts_types_name,NaN,linkColor
ts_types_optional,NaN,True
ts_types_py_name,NaN,link_color
ts_types_origin_name,NaN,GraphConfigInterface


In [143]:
differences_for_name('link_color_by')


,current,stage
ts_types_description,NaN,Column name for the link color. Links will be ...
ts_types_type,NaN,string
ts_types_name,NaN,linkColorBy
ts_types_optional,NaN,True
ts_types_py_name,NaN,link_color_by
ts_types_origin_name,NaN,CosmographLinksConfig


## Replacing current with stage (a.k.a. updating the interface)

In [31]:
list(current.config_jsons)

['config_info.json',
 '_color_table.parquet',
 '_widget_config_from_md.json',
 'config_prep/parsed_types.json',
 'config_prep/source_strings.json',
 'config_prep/parsed_defaults.json',
 'config_prep/parsed_descriptions.json',
 'color_names.json',
 '_widget_config.json',
 'nikita_config.json',
 'cosmos_config.json']

In [7]:
list(stage.config_jsons)

['config_prep/parsed_types.json',
 'config_prep/source_strings.json',
 'config_prep/parsed_defaults.json',
 'config_prep/parsed_descriptions.json']

In [8]:
## Uncomment this and run to update current files with stage ones:
# current.config_jsons.update(stage.config_jsons)

## Recreating and replacing the params_ssot.json file

In [137]:
new_params_ssot = stage.cosmograph_base_params_json()
print(new_params_ssot[:300])

[
    {
        "name":"disable_simulation",
        "default":false,
        "annotation":"bool",
        "description":"Prevents the simulation from running, merely rendering the graph."
    },
    {
        "name":"simulation_decay",
        "default":1000,
        "annotation":"float",
        "


In [139]:
# TODO: Add some dict-diff diagnosis on the existing params_ssot.json

In [138]:
from cosmograph.util import PARAMS_SSOT_PATH

# Uncomment this and run to update the params_ssot file with the new one: CAREFUL! BE SURE YOU WANT TO DO THIS!
# PARAMS_SSOT_PATH.write_text(new_params_ssot)

15070

In [146]:
from cosmograph.util import cosmograph_base_signature, cosmograph_base_docs

cosmograph_base_signature()

<Sig (*, disable_simulation: bool = False, simulation_decay: float = 1000, simulation_gravity: float = 0, simulation_center: float = 0, simulation_repulsion: float = 0.1, simulation_repulsion_theta: float = 1.7, simulation_repulsion_quadtree_levels: float = 12, simulation_link_spring: float = 1, simulation_link_distance: float = 2, simulation_link_dist_random_variation_range: list[typing.Any] = [1, 1.2], simulation_repulsion_from_mouse: float = 2, simulation_friction: float = 0.85, simulation_cluster: float = None, background_color: Union[str, list[float]] = '#222222', space_size: int = 4096, point_color: Union[str, list[float]] = '#b3b3b3', point_greyout_opacity: float = 0.1, point_size: float = 4, point_size_scale: float = 1, hovered_point_cursor: str = None, render_hovered_point_ring: bool = 0.7, hovered_point_ring_color: Union[str, list[float]] = 'white', focused_point_ring_color: Union[str, list[float]] = 0.95, focused_point_index: int = None, render_links: bool = True, link_color

In [147]:
cosmograph_base_docs()

'Parameters\n----------\ndisable_simulation : bool, default=False\n    Prevents the simulation from running, merely rendering the graph.\nsimulation_decay : float, default=1000\n    Defines how quickly the simulation cools down.\nsimulation_gravity : float, default=0\n    Coefficient for gravity force.\nsimulation_center : float, default=0\n    Centers the mass force coefficient.\nsimulation_repulsion : float, default=0.1\n    Configures point repulsion between points.\nsimulation_repulsion_theta : float, default=1.7\n    Decreases / increases the detalization of the Many-Body force calculations.\nsimulation_repulsion_quadtree_levels : float, default=12\n    Barnes–Hut approximation depth, usable when useQuadtree is set to True.\nsimulation_link_spring : float, default=1\n    Spring constant for links.\nsimulation_link_distance : float, default=2\n    Default distance for links.\nsimulation_link_dist_random_variation_range : list[typing.Any], default=[1, 1.2]\n    Random link distance 

# Code gen

In [18]:
from cosmograph._code_sync import code_str_with_signature
from cosmograph.util import cosmograph_base_signature
# from cosmograph.base import cosmo 
from i2 import Sig

cosmo_base_sig = cosmograph_base_signature()
cosmo_base_sig

from inspect import getsource

old_code_str = getsource(foo)

cosmo_sig = Sig('(data=None)').merge_with_sig(cosmo_base_sig)

new_code_str = code_str_with_signature(cosmo, cosmo_sig)
print(new_code_str)


def foo_with_new_sig(
    data=None,
    *,
    disable_simulation: bool = False,
    simulation_decay: float = 1000,
    simulation_gravity: float = 0,
    simulation_center: float = 0,
    simulation_repulsion: float = 0.1,
    simulation_repulsion_theta: float = 1.7,
    simulation_repulsion_quadtree_levels: float = 12,
    simulation_link_spring: float = 1,
    simulation_link_distance: float = 2,
    simulation_link_dist_random_variation_range: list[typing.Any] = [1, 1.2],
    simulation_repulsion_from_mouse: float = 2,
    simulation_friction: float = 0.85,
    simulation_cluster: float = None,
    background_color: Union[str, list[float]] = '#222222',
    space_size: int = 4096,
    point_color: Union[str, list[float]] = '#b3b3b3',
    point_greyout_opacity: float = 0.1,
    point_size: float = 4,
    point_size_scale: float = 1,
    hovered_point_cursor: str = None,
    render_hovered_point_ring: bool = 0.7,
    hovered_point_ring_color: Union[str, list[float]] = 'white',
    f

In [5]:
from cosmograph._code_sync import code_str_with_signature

def original_function(a, b=10, *args, **kwargs):
    """
    This is a test function to demonstrate the functionality of
    `code_str_with_signature`. It adds `a` and `b`, and prints any
    additional arguments.

    Parameters
    ----------
    a : int
        The first number to add.
    b : int, optional
        The second number to add (default is 10).
    *args : tuple
        Additional positional arguments.
    **kwargs : dict
        Additional keyword arguments.

    Returns
    -------
    int
        The sum of `a` and `b`.
    """
    result = a + b
    if args:
        print("Positional args:", args)
    if kwargs:
        print("Keyword args:", kwargs)
    return result

# Custom signature to apply
new_signature = """def modified_function(x, y=20, z=None, *args, custom_kwarg='default', **kwargs):"""

# Generate the new function code as a string
new_function_code = code_str_with_signature(original_function, new_signature)

# Print the generated code to inspect it
print("Generated Code:\n")
print(new_function_code)

# Execute the generated code to test it dynamically
namespace = {}
exec(new_function_code, namespace)
modified_function = namespace["modified_function"]

# Tests
print("\nRunning Tests:")
print(modified_function(5, 15))  # Simple addition, no extra args
print(modified_function(3, 7, 1, 2, 3, custom_kwarg="test"))  # Positional and keyword args

Generated Code:

def modified_function(
    x,
    y=20,
    z=None,
    *args,
    custom_kwarg='default',
    **kwargs
):
    """This is a test function to demonstrate the functionality of
`code_str_with_signature`. It adds `a` and `b`, and prints any
additional arguments.

Parameters
----------
a : int
    The first number to add.
b : int, optional
    The second number to add (default is 10).
*args : tuple
    Additional positional arguments.
**kwargs : dict
    Additional keyword arguments.

Returns
-------
int
    The sum of `a` and `b`."""
    x, y, z, args, custom_kwarg, kwargs = locals().values()
    result = a + b
    if args:
        print("Positional args:", args)
    if kwargs:
        print("Keyword args:", kwargs)
    return result

Running Tests:


NameError: name 'a' is not defined

In [6]:
modified_function
print(new_function_code)

def modified_function(
    x,
    y=20,
    z=None,
    *args,
    custom_kwarg='default',
    **kwargs
):
    """This is a test function to demonstrate the functionality of
`code_str_with_signature`. It adds `a` and `b`, and prints any
additional arguments.

Parameters
----------
a : int
    The first number to add.
b : int, optional
    The second number to add (default is 10).
*args : tuple
    Additional positional arguments.
**kwargs : dict
    Additional keyword arguments.

Returns
-------
int
    The sum of `a` and `b`."""
    x, y, z, args, custom_kwarg, kwargs = locals().values()
    result = a + b
    if args:
        print("Positional args:", args)
    if kwargs:
        print("Keyword args:", kwargs)
    return result


# ResourcesDacc

`ResourcesDacc` is a facade to resources datas that are stored locally. 

In [ ]:
from cosmograph._resources import _get_fresh_color_table, ResourcesDacc

resources = ResourcesDacc()

You can see what resources files you have by listing the `resources_jsons`.

In [ ]:
list(resources.resources_jsons)

['config_info.json',
 '_color_table.parquet',
 '_widget_config_from_md.json',
 'config_prep/parsed_types.json',
 'config_prep/source_strings.json',
 'config_prep/parsed_defaults.json',
 'config_prep/parsed_descriptions.json',
 'color_names.json',
 '_widget_config.json',
 'nikita_config.json',
 'cosmos_config.json']

Examples of accessing (formatted/prepared) resource data via attributes:

In [ ]:
print(f"{resources._color_table.shape=}")
resources._color_table.iloc[0]

resources._color_table.shape=(148, 3)


color_name      aliceblue
hex_rgb           #f0f8ff
decimal       240 248 255
Name: 0, dtype: object

In [ ]:
print(f"{len(resources.color_names)=}")
resources.color_names[:5]

len(resources.color_names)=148


['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure']

# Parsing TS

In [ ]:
# testing out the parse_ts
from jy.ts_parse import test_parse_ts
test_parse_ts()  # issues warning, but should not update (this works with tree-siter 0.21.3)

/Users/thorwhalen/.pyenv/versions/3.10.13/envs/p10/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [ ]:
from graze import graze
from jy import parse_ts, parse_ts_with_oa




# Trying out cosmograph_widget.Cosmograph

In [ ]:
from cosmograph_widget import Cosmograph
import pandas as pd
# dir(Cosmograph)

In [ ]:
points = pd.read_json('./points.json')
points.head(3)

,id,index,value,value2,date,time,color,label
0,0,0,-5,116,2022-11-25 03:25:04.851000+00:00,1970-01-01T01:20:59.956Z,#bab0ab,offence
1,1,1,17,155,2020-08-21 06:38:36.826000+00:00,1970-01-01T00:40:26.814Z,#9c755f,analytics
2,2,2,-31,51,2012-05-15 07:34:20.927000+00:00,1970-01-01T01:07:34.760Z,#9c755f,minor-league


In [ ]:
links = pd.read_json('./links.json')
links.head(2)

,source,sourceidx,date,time,color,target,targetidx
0,0,0,2012-02-08 11:17:27.900000+00:00,1970-01-01T01:06:54.149Z,#4e79a7,1,1
1,0,0,2012-02-08 11:17:27.900000+00:00,1970-01-01T01:06:54.149Z,#4e79a7,92,92


In [ ]:
cosmo = Cosmograph(points=points, links=links,
           point_id='id',
           point_index='index',
           point_color='color',
           point_label='label',
           point_include_columns=['value', 'value2'],
           link_source='source',
           link_source_index='sourceidx',
           link_target='target',
           link_target_index='targetidx',
           link_color='color',
           link_width='value'
        )
cosmo

Cosmograph(background_color=None, default_link_color=None, default_point_color=None, focused_point_ring_color=…

In [ ]:
cosmo.activate_rect_selection()

In [ ]:
cosmo.deactivate_rect_selection()

In [ ]:
cosmo.selected_point_indices

[5482, 5693, 6338, 6430]

In [ ]:
cosmo.select_points_by_indices([1,2,3,4,5])
# cosmo.select_points_by_indices([]) # unselect points

In [ ]:
cosmo.focus_point(5)
# cosmo.focus_point()

In [ ]:
# cosmo.fit_view()
# cosmo.fit_view_by_indices([1,2,3,4,5], 250, 0.1)
cosmo.fit_view_by_coordinates([1400, 1400, 2500, 2500], 2000)

In [ ]:
cosmo.background_color = 'hotpink'
# cosmo.background_color = 'gold'

# Gathering widget properties

In [ ]:
from cosmograph_widget import Cosmograph

vars(Cosmograph)

mappingproxy({'__module__': 'cosmograph_widget',
              '_esm': <anywidget._file_contents.FileContents at 0x1313d2bf0>,
              '_css': <anywidget._file_contents.FileContents at 0x1313d2ad0>,
              'disable_simulation': <traitlets.traitlets.Bool at 0x107ea52a0>,
              'simulation_decay': <traitlets.traitlets.Float at 0x107edc070>,
              'simulation_gravity': <traitlets.traitlets.Float at 0x131346e30>,
              'simulation_center': <traitlets.traitlets.Float at 0x131346e00>,
              'simulation_repulsion': <traitlets.traitlets.Float at 0x131346dd0>,
              'simulation_repulsion_theta': <traitlets.traitlets.Float at 0x1205cbd30>,
              'simulation_repulsion_quadtree_levels': <traitlets.traitlets.Float at 0x1205cbe20>,
              'simulation_link_spring': <traitlets.traitlets.Float at 0x1205cbe50>,
              'simulation_link_distance': <traitlets.traitlets.Float at 0x1313d1690>,
              'simulation_link_dist_rando

[cosmograph_widget.Cosmograph,
 anywidget.widget.AnyWidget,
 ipywidgets.widgets.domwidget.DOMWidget,
 ipywidgets.widgets.widget.Widget,
 ipywidgets.widgets.widget.LoggingHasTraits,
 traitlets.traitlets.HasTraits,
 traitlets.traitlets.HasDescriptors,
 object]

In [ ]:
w = [1,2,3,4, 5]
w[::-1][:-1]
# different way of doing w[::-1][:-1]
w[-1:0:-1]

[5, 4, 3, 2]

In [ ]:
def vars_parents_diff(typ):
    parent_types = typ.mro()[-1:0:-1]
    parent_vars = {}
    for parent in parent_types:
        parent_vars.update(vars(parent))

    return {k: v for k, v in vars(typ).items() if k not in parent_vars}


t = vars_parents_diff(Cosmograph)
t = {k: v for k, v in t.items() if not callable(v)}
t


{'_esm': <anywidget._file_contents.FileContents at 0x1313d2bf0>,
 '_css': <anywidget._file_contents.FileContents at 0x1313d2ad0>,
 'disable_simulation': <traitlets.traitlets.Bool at 0x107ea52a0>,
 'simulation_decay': <traitlets.traitlets.Float at 0x107edc070>,
 'simulation_gravity': <traitlets.traitlets.Float at 0x131346e30>,
 'simulation_center': <traitlets.traitlets.Float at 0x131346e00>,
 'simulation_repulsion': <traitlets.traitlets.Float at 0x131346dd0>,
 'simulation_repulsion_theta': <traitlets.traitlets.Float at 0x1205cbd30>,
 'simulation_repulsion_quadtree_levels': <traitlets.traitlets.Float at 0x1205cbe20>,
 'simulation_link_spring': <traitlets.traitlets.Float at 0x1205cbe50>,
 'simulation_link_distance': <traitlets.traitlets.Float at 0x1313d1690>,
 'simulation_link_dist_random_variation_range': <traitlets.traitlets.List at 0x1313d1660>,
 'simulation_repulsion_from_mouse': <traitlets.traitlets.Float at 0x1313d16c0>,
 'simulation_friction': <traitlets.traitlets.Float at 0x1313d1

In [ ]:
from graze import graze

url = 'https://raw.githubusercontent.com/cosmograph-org/cosmograph/refs/heads/dev/packages/cosmograph/src/modules/cosmograph/config/simulation.ts?token=GHSAT0AAAAAACSGWAOWSPSYWQEN2BYDRHO6ZYH5S6Q'
code_str = graze(url).decode()
len(code_str)


import jy

t = jy.func_name_and_params_pairs(code_str)

In [ ]:
code_str

b'import { GraphSimulationSettings } from \'@cosmograph/cosmos\'\n\nexport interface SimulationConfig {\n  /**\n   * Decay coefficient. Use bigger values if you want the simulation to "cool down" slower.\n   * Default value: `1000`\n   */\n  simulationDecay?: GraphSimulationSettings[\'decay\'];\n\n  /**\n   * Gravity force coefficient.\n   * Default value: `0`\n   */\n  simulationGravity?: GraphSimulationSettings[\'gravity\'];\n\n  /**\n   * Centering to center mass force coefficient.\n   * Default value: `0`\n   */\n  simulationCenter?: GraphSimulationSettings[\'center\'];\n\n  /**\n   * Repulsion force coefficient.\n   * Default value: `0.1`\n   */\n  simulationRepulsion?: GraphSimulationSettings[\'repulsion\'];\n\n  /**\n   * Decreases / increases the detalization of the Many-Body force calculations.\n   * When `useQuadtree` is set to `true`, this property corresponds to the Barnes\xe2\x80\x93Hut approximation criterion.\n   * Default value: `1.7`\n   */\n  simulationRepulsionTheta?

In [ ]:
import pyperclip
pyperclip.copy(code_str)

In [ ]:
print(code_str)

import { GraphSimulationSettings } from '@cosmograph/cosmos'

export interface SimulationConfig {
  /**
   * Decay coefficient. Use bigger values if you want the simulation to "cool down" slower.
   * Default value: `1000`
   */
  simulationDecay?: GraphSimulationSettings['decay'];

  /**
   * Gravity force coefficient.
   * Default value: `0`
   */
  simulationGravity?: GraphSimulationSettings['gravity'];

  /**
   * Centering to center mass force coefficient.
   * Default value: `0`
   */
  simulationCenter?: GraphSimulationSettings['center'];

  /**
   * Repulsion force coefficient.
   * Default value: `0.1`
   */
  simulationRepulsion?: GraphSimulationSettings['repulsion'];

  /**
   * Decreases / increases the detalization of the Many-Body force calculations.
   * When `useQuadtree` is set to `true`, this property corresponds to the Barnes–Hut approximation criterion.
   * Default value: `1.7`
   */
  simulationRepulsionTheta?: GraphSimulationSettings['repulsionTheta'];

  /**
   

In [ ]:
# TODO: Delete when integrated to config_properties.py

# from oa import chat
# from oa.util import data_files

# # To make sure we get a json schema that is openAI compliant, we'll use an example of 
# # one in our prompt to AI to give us one...
# example_of_a_openai_json_schema = example_of_a_openai_json_schema = (
#     data_files.joinpath('json_schema_example.json').read_text()
# )

# json_schema_str = chat("""
#     Give me the json of a json_schema I can use to represent information on the objects 
#     that you'll parse out of typescript definitions. 
    
#     For example, if given a TS string like this:
                       
#     ```
# import { GraphSimulationSettings } from '@cosmograph/cosmos'

# export interface SimulationConfig {
#   /**
#    * Decay coefficient. Use bigger values if you want the simulation to "cool down" slower.
#    * Default value: `1000`
#    */
#   simulationDecay?: GraphSimulationSettings['decay'];

#   /**
#    * Gravity force coefficient.
#    * Default value: `0`
#    */
#   simulationGravity?: GraphSimulationSettings['gravity'];
                       
# export interface SimulationEventConfig {
#   /** Callback function that will be called when the simulation starts. */
#   onSimulationStart?: GraphSimulationSettings['onStart'];
                       
# I'd like my json to communicate the fact that there were two object (SimulationConfig and SimulationEventConfig)
# that wer interfaces, and also give me the information about those object's properties
# for each, their name, type (if available), decription (if available), default (if available), 
# and what ever you think might be useful
# ```
# """,
#     model='gpt-4o-mini',
#     response_format={'type': 'json_object'}
# )

# import json
# from pprint import pprint

# schema = json.loads(json_schema_str)

# schema

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'type': 'object',
 'properties': {'interfaces': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the interface.'},
     'description': {'type': 'string',
      'description': 'A brief description of the interface (if available).'},
     'properties': {'type': 'array',
      'items': {'type': 'object',
       'properties': {'name': {'type': 'string',
         'description': 'The name of the property within the interface.'},
        'type': {'type': 'string',
         'description': 'The TypeScript type of the property.'},
        'description': {'type': 'string',
         'description': 'A brief description of the property (if available).'},
        'default': {'type': ['string', 'number', 'null'],
         'description': 'The default value for the property (if available).'},
        'optional': {'type': 'boolean',
         'description': 'Indicates whe

In [ ]:
# TODO: Delete when integrated to config_properties.py


# from oa import prompt_function
# import json
# from typing import Mapping

# from functools import partial

# from oa.tools import prompt_json_function

# schema = {
#     'name': 'whatevs',
#     'strict': False,
#     'schema': {
#         'type': 'object',
#         'properties': {
#             'interfaces': {
#                 'type': 'array',
#                 'items': {
#                     'type': 'object',
#                     'properties': {
#                         'name': {
#                             'type': 'string',
#                             'description': 'The name of the interface.',
#                         },
#                         'description': {
#                             'type': 'string',
#                             'description': 'A brief description of the interface (if available).',
#                         },
#                         'properties': {
#                             'type': 'array',
#                             'items': {
#                                 'type': 'object',
#                                 'properties': {
#                                     'name': {
#                                         'type': 'string',
#                                         'description': 'The name of the property within the interface.',
#                                     },
#                                     'type': {
#                                         'type': 'string',
#                                         'description': 'The TypeScript type of the property.',
#                                     },
#                                     'description': {
#                                         'type': 'string',
#                                         'description': 'A brief description of the property (if available).',
#                                     },
#                                     'default': {
#                                         'type': ['string', 'number', 'null'],
#                                         'description': 'The default value for the property (if available).',
#                                     },
#                                     'optional': {
#                                         'type': 'boolean',
#                                         'description': 'Indicates whether the property is optional.',
#                                     },
#                                 },
#                                 'required': ['name'],
#                             },
#                         },
#                     },
#                     'required': ['name', 'properties'],
#                 },
#             }
#         },
#     }
# }


# from graze import graze 


# url = 'https://raw.githubusercontent.com/cosmograph-org/cosmograph/refs/heads/dev/packages/cosmograph/src/modules/cosmograph/config/simulation.ts?token=GHSAT0AAAAAACSGWAOWSPSYWQEN2BYDRHO6ZYH5S6Q'
# code_str = graze(url).decode()

# parse_my_ts = prompt_json_function(
#     """
#     You are a typescript parser.
#     Parse through the following typescript file(s) contents and extract information 
#     about the objects in them, returning a json that fits the json_schema.

#     {ts_code}
#     """,
#     json_schema=schema,
# )

# t = parse_my_ts(code_str)

In [ ]:
# TODO: Delete when integrated to config_properties.py


# from oa import prompt_function
# import json
# from typing import Mapping

# from functools import partial

# from oa.tools import prompt_json_function

# schema = {
#     'name': 'whatevs',
#     'strict': False,
#     'schema': {
#         'type': 'object',
#         'properties': {
#             'interfaces': {
#                 'type': 'array',
#                 'items': {
#                     'type': 'object',
#                     'properties': {
#                         'name': {
#                             'type': 'string',
#                             'description': 'The name of the interface.',
#                         },
#                         'description': {
#                             'type': 'string',
#                             'description': 'A brief description of the interface (if available).',
#                         },
#                         'properties': {
#                             'type': 'array',
#                             'items': {
#                                 'type': 'object',
#                                 'properties': {
#                                     'name': {
#                                         'type': 'string',
#                                         'description': 'The name of the property within the interface.',
#                                     },
#                                     'type': {
#                                         'type': 'string',
#                                         'description': 'The type of the property.',
#                                     },
#                                     'description': {
#                                         'type': 'string',
#                                         'description': 'A brief description of the property (if available).',
#                                     },
#                                     'default': {
#                                         'type': ['string', 'number', 'null'],
#                                         'description': 'The default value for the property (if available).',
#                                     },
#                                     'group': {
#                                         'type': ['string'],
#                                         'description': 'The group to which the property belongs.',
#                                     },
#                                     'optional': {
#                                         'type': 'boolean',
#                                         'description': 'Indicates whether the property is optional.',
#                                     },
#                                 },
#                                 'required': ['name', 'default'],
#                             },
#                         },
#                     },
#                     'required': ['name', 'properties'],
#                 },
#             }
#         },
#     }
# }


# from graze import graze 


# parse_my_md = prompt_json_function(
#     """
#     Parse through the following markdown contents and extract information 
#     about the objects in them, returning a json that fits the json_schema.
#     The "group" field is supposed to denote the section where you found the property;
#     i.e. "Minimal Configuration for Points", "Additional Points Configuration", etc.

#     You must always include a default field in the properties. 
#     It is usually in the descriptions as "Default value: ...". 
#     If you can't find a default, please just set it to "NO_DEFAULT", but still include the field.

#     {md_string}
#     """,
#     json_schema=schema,
# )



# md_string = """## Data and Configuration Parameters

# The Cosmograph widget builds upon the native Cosmograph library, inheriting most of its configuration parameters, most of which are optional and can be modified in real-time. For detailed insights about configuration parameters, refer to the [documentation](../lib/api/interfaces/CosmographConfig.mdx).

# ### Minimal Configuration for Points

# To render a graph, the following minimal configuration is required:

# - **`points`**: Data in a pandas DataFrame format.
# - **`point_id`**: Unique identifier column for each point.
# - **`point_index`**: Ordinal index of each point ranging from 0 to x (the number of unique points).

# ### Additional Points Configuration

# - **`point_color`**: Column name for point colors.
# - **`point_size`**: Column name for point sizes.
# - **`point_label`**: Column name for associated point labels.
# - **`point_label_weight`**: Column name for point label weights.
# - **`point_x`**: Column name for the x-coordinate.
# - **`point_y`**: Column name for the y-coordinate.
# - **`point_include_columns`**: An array of additional column names to include in point data.

# ### Minimal Configuration for Links

# To define links, the following fields are necessary:

# - **`links`**: Data in a pandas DataFrame format.
# - **`link_source`**: Column with the unique identifier for source points.
# - **`link_source_index`**: Index column of the source point, corresponding to `point_index`.
# - **`link_target`**: Column with the unique identifier for target points.
# - **`link_target_index`**: Index column of the target point, corresponding to `point_index`.

# ### Additional Links Configuration

# - **`link_color`**: Column name for link colors.
# - **`link_width`**: Column name for link widths.
# - **`link_arrow`**: Column name that indicates links with arrows.
# - **`link_strength`**: Column name for link strengths.
# - **`link_include_columns`**: An array of additional column names to include in link data.

# ### Simulation Settings

# Various simulation settings allow for detailed customization:

# - **`disable_simulation`**: Prevents the simulation from running, merely rendering the graph. Default: `None`.
# - **`simulation_decay`**: Defines how quickly the simulation cools down. Default: `1000`.
# - **`simulation_gravity`**: Coefficient for gravity force. Default: `0`.
# - **`simulation_center`**: Centers the mass force coefficient. Default: `0`.
# - **`simulation_repulsion`**: Configures point repulsion between points. Default: `0.1`.
# - **`simulation_repulsion_theta`**: Decreases / increases the detalization of the Many-Body force calculations. When `useQuadtree` is set to `True`, this property corresponds to the Barnes–Hut approximation criterion. Default: `1.7`.
# - **`simulation_link_spring`**: Spring constant for links. Default value: `1`.
# - **`simulation_link_distance`**: Default distance for links. Default value: `2`.
# - **`simulation_link_dist_random_variation_range`**: Random link distance range. Default value: `[1, 1.2]`.
# - **`simulation_repulsion_from_mouse`**:  Mouse position repulsion coefficient, activated by right-click. Default value: `2`.
# - **`simulation_friction`**: Sets simulation friction. Default value: `0.85`.

# ### Quadtree algorithm settings

# - **`use_quadtree`**: Activates quadtree algorithm for Many-Body force when set to `True`. Default value: `False`.
# - **`simulation_repulsion_quadtree_levels`**: Barnes–Hut approximation depth, usable when `useQuadtree` is set to `True`. Default value: `12`.

# ### Graph Appearance

# Customize the widget’s visual aspects:

# - **`background_color`**: Canvas background color. Default value: `'#222222'`.
# - **`space_size`**: Size of the simulation space. Default value: `4096`.
# - **`default_point_color`**: Default point size when `point_color` not specified. Default value: `'#b3b3b3'`.
# - **`point_greyout_opacity`**: Opacity of unselected nodes during selection. Default value: `0.1`.
# - **`default_point_size`**: The default size value to use for points when no `point_size` are provided. Default: `4`.
# - **`point_size_scale`**: Scale factor for point sizes. Default value: `1`.
# - **`hovered_point_cursor`**: Cursor type when hovering over a point. Default value: `'auto'`.
# - **`render_hovered_point_ring`**: Enables ring around hovered points. Default value: `False`.
# - **`hovered_point_ring_color`**: Color of hovered point ring. Default value: `'white'`.
# - **`focused_point_ring_color`**: Color of the focused point ring. Default value: `'white'`.
# - **`focused_point_index`**: Index of the focused point, prioritized over `focus_point` method. Default value: `None`.
# - **`render_links`**: Enables or disables link rendering. Default value: `True`.
# - **`default_link_color`**: Default link color when `link_color` not specified. Default value: `'#666666'`.
# - **`link_greyout_opacity`**: Opacity of unselected links during selection. Default value: `0.1`.
# - **`default_link_width`**: Default link width when `link_width` not specified. Default value: `1`.
# - **`link_width_scale`**: Scale factor for link widths. Default value: `1`.
# - **`curved_links`**: Enables or disables curved links. Default value: `False`.
# - **`curved_link_segments`**: Segments defining curved links. Default value: `19`.
# - **`curved_link_weight`**: Weight factor for link curvature. Default value: `0.8`.
# - **`curved_link_control_point_distance`**: Control point positioning for curves. If set to 1 then the control point is at a distance equal to the length of the line. Default value: `0.5`.
# - **`default_link_arrows`**: Enables or disables arrows on links when `link_arrow` not specified. Default value: `False`.
# - **`link_arrows_size_scale`**: Scale factor for link arrow size. Default value: `1`.
# - **`link_visibility_distance_range`**: Pixel distance range for link transparency. Default value: `[50, 150]`.
# - **`link_visibility_min_transparency`**: Minimum transparency of links based on `link_visibility_distance_range`. Default value: `0.25`.
# - **`scale_points_on_zoom`**: Scales point sizes when zooming. Default value: `True`.
# - **`initial_zoom_level`**: Starting zoom level. Default value: `None`.
# - **`disable_zoom`**: Enables or disables zooming. Default value: `False`.
# - **`enable_drag`**: Allows graph dragging. Default value: `False`.
# - **`fit_view_on_init`**: Automatically fits view to all points upon initialization. Default value: `True`.
# - **`fit_view_delay`**: Delay for fitting view after initialization in milliseconds. Default value: `250`.
# - **`fit_view_padding`**: Padding around fit view area. Default value: `0.1`.
# - **`fit_view_duration`**: Animation duration for view fitting in milliseconds. Default value: `250`.
# - **`fit_view_by_points_in_rect`**: Fits view to specified rectangle of points, active when `fit_view_on_init` is `True`. Default value: `None`.

# ### Label Options

# Control label display on the graph:

# - **`show_dynamic_labels`**: Flag to show dynamic labels for visible points. Default value: `False`.
# - **`show_labels_for`**: An array of point ids for which to show labels. Default value: `None`.
# - **`show_top_labels`**: Flag to display labels for the top points. Default values: `False`.
# - **`show_top_labels_limit`**: Maximum number of top points to show labels for. Default value: `100`.
# - **`show_top_labels_by`**: Column to determine which points are considered as a top. If not provided, the top points will be sorted by their total links count. Default value: `None`.
# - **`static_label_weight`**: Weight of static labels. Default value: `0.8`.
# - **`dynamic_label_weight`**: Weight of dynamic labels. Default value: `0.7`.
# - **`label_padding`**: Padding around labels in the graph.
# - **`show_hovered_point_label`**: Flag to display the label for the currently hovered point. Default value: `False`.

# ### Additional Parameters

# Various additional parameters enhance functionality:

# - **`show_FPS_monitor`**: Display an FPS counter in the upper right corner of the canvas. Default value: `False`.
# - **`pixel_ratio`**: Canvas pixel ratio. Default value: `2`.
# - **`random_seed`**: Seed value for generating random numbers in simulations. Default value: `None`.
# - **`point_sampling_distance`**: Distance threshold for sampling points. Default value: `150`.

# ## Methods and Values

# The Cosmograph widget provides several methods for data interaction:

# - **`select_point_by_index(index)`**: Selects point by index.
# - **`select_points_by_indices(indices)`**: Selects points by indices.
# - **`activate_rect_selection()`**: Activates rectangular selection.
# - **`deactivate_rect_selection()`**: Deactivates rectangular selection.
# - **`fit_view()`**: Center and zoom in/out the view to fit all points in the scene.
# - **`fit_view_by_indices(indices, duration, padding)`**: Center and zoom in/out the viewport to fit points by their indices.
# - **`fit_view_by_coordinates(coordinates, duration, padding)`**: Fit the given coordinates into the viewport.
# - **`focus_point(index)`**: Set focus on a point by index. A ring will be drawn around the focused point.
# - **`start(alpha)`**: Starts the simulation.
# - **`pause()`**: Pause the simulation.
# - **`restart()`**: Restarts the simulation.
# - **`step()`**: Render only one frame of the simulation.

# #### Values

# - **`clicked_point_index`**: Point index that currently clicked.
# - **`selected_point_indices`**: Point indices that currently selected.

# #### Legends
# - **`disable_point_color_legend`**: Disable point color legend.
# - **`disable_point_size_legend`**: Disable point size legend.
# - **`disable_link_color_legend`**: Disable link color legend.
# - **`disable_link_width_legend`**: Disable link width legend."""


# parsed_md_string = parse_my_md(md_string)

# # parsed_md_dict = json.loads(parsed_md_string)
# print(json.dumps(parsed_md_string, indent=2))


{
  "interfaces": [
    {
      "name": "CosmographConfig",
      "description": "The configuration parameters for the Cosmograph widget.",
      "properties": [
        {
          "name": "points",
          "type": "DataFrame",
          "description": "Data in a pandas DataFrame format.",
          "group": "Minimal Configuration for Points",
          "optional": false,
          "default": "NO_DEFAULT"
        },
        {
          "name": "point_id",
          "type": "string",
          "description": "Unique identifier column for each point.",
          "group": "Minimal Configuration for Points",
          "optional": false,
          "default": "NO_DEFAULT"
        },
        {
          "name": "point_index",
          "type": "integer",
          "description": "Ordinal index of each point ranging from 0 to x (the number of unique points).",
          "group": "Minimal Configuration for Points",
          "optional": false,
          "default": "NO_DEFAULT"
        },
   

In [ ]:
{
    'interfaces': [
        {
            'name': 'SimulationConfig',
            'description': 'Configuration options for the simulation settings.',
            'properties': [
                {
                    'name': 'simulationDecay',
                    'type': "GraphSimulationSettings['decay']",
                    'description': "Decay coefficient. Use bigger values if you want the simulation to 'cool down' slower. Default value: `1000`",
                    'optional': True,
                },
                {
                    'name': 'simulationGravity',
                    'type': "GraphSimulationSettings['gravity']",
                    'description': 'Gravity force coefficient. Default value: `0`',
                    'optional': True,
                },
                {
                    'name': 'simulationCenter',
                    'type': "GraphSimulationSettings['center']",
                    'description': 'Centering to center mass force coefficient. Default value: `0`',
                    'optional': True,
                },
                {
                    'name': 'simulationRepulsion',
                    'type': "GraphSimulationSettings['repulsion']",
                    'description': 'Repulsion force coefficient. Default value: `0.1`',
                    'optional': True,
                },
                {
                    'name': 'simulationRepulsionTheta',
                    'type': "GraphSimulationSettings['repulsionTheta']",
                    'description': 'Decreases / increases the detalization of the Many-Body force calculations. When `useQuadtree` is set to `true`, this property corresponds to the Barnes–Hut approximation criterion. Default value: `1.7`',
                    'optional': True,
                },
                {
                    'name': 'simulationRepulsionQuadtreeLevels',
                    'type': "GraphSimulationSettings['repulsionQuadtreeLevels']",
                    'description': 'Barnes–Hut approximation depth. Can only be used when `useQuadtree` is set `true`. Default value: `12`',
                    'optional': True,
                },
                {
                    'name': 'simulationLinkSpring',
                    'type': "GraphSimulationSettings['linkSpring']",
                    'description': 'Link spring force coefficient. Default value: `1`',
                    'optional': True,
                },
                {
                    'name': 'simulationLinkDistance',
                    'type': "GraphSimulationSettings['linkDistance']",
                    'description': 'Minimum link distance. Default value: `2`',
                    'optional': True,
                },
                {
                    'name': 'simulationLinkDistRandomVariationRange',
                    'type': "GraphSimulationSettings['linkDistRandomVariationRange']",
                    'description': 'Range of random link distance values. Default value: `[1, 1.2]`',
                    'optional': True,
                },
                {
                    'name': 'simulationRepulsionFromMouse',
                    'type': "GraphSimulationSettings['repulsionFromMouse']",
                    'description': 'Repulsion coefficient from mouse position. The repulsion force is activated by pressing the right mouse button. Default value: `2`',
                    'optional': True,
                },
                {
                    'name': 'simulationFriction',
                    'type': "GraphSimulationSettings['friction']",
                    'description': 'Friction coefficient. Default value: `0.85`',
                    'optional': True,
                },
            ],
        },
        {
            'name': 'SimulationEventConfig',
            'description': 'Configuration options for simulation event callbacks.',
            'properties': [
                {
                    'name': 'onSimulationStart',
                    'type': "GraphSimulationSettings['onStart']",
                    'description': 'Callback function that will be called when the simulation starts.',
                    'optional': True,
                },
                {
                    'name': 'onSimulationEnd',
                    'type': "GraphSimulationSettings['onEnd']",
                    'description': 'Callback function that will be called when the simulation stops.',
                    'optional': True,
                },
                {
                    'name': 'onSimulationPause',
                    'type': "GraphSimulationSettings['onPause']",
                    'description': 'Callback function that will be called when the simulation is paused.',
                    'optional': True,
                },
                {
                    'name': 'onSimulationRestart',
                    'type': "GraphSimulationSettings['onRestart']",
                    'description': 'Callback function that will be called when the simulation is restarted.',
                    'optional': True,
                },
                {
                    'name': 'onSimulationTick',
                    'type': "GraphSimulationSettings['onTick']",
                    'description': 'Callback function that will be called on every tick of the simulation.',
                    'optional': True,
                },
            ],
        },
    ]
}

{'args': ('\n    You are a typescript parser.\n    Parse through the following typescript file(s) contents and extract information \n    about the objects in them, returning a json that fits the json_schema.\n\n    {ts_code}\n    ',),
 'kwargs': {}}

In [ ]:
{'simpleProp': {'name': 'simpleProp', 'optional': False, 'type': ': string', 'kind': 'property'}, 'optionalProp': {'name': 'optionalProp', 'optional': False, 'type': ': number', 'kind': 'property'}, 'functionTypeProp': {'name': 'functionTypeProp', 'optional': False, 'type': ': (x: T) => boolean', 'kind': 'function', 'parameters': [{'type': ': T', 'optional': False, 'rest': False}]}, 'arrayProp': {'name': 'arrayProp', 'optional': False, 'type': ': number[]', 'kind': 'property'}, 'unionProp': {'name': 'unionProp', 'optional': False, 'type': ': string | number', 'kind': 'property'}, 'nested': {'name': 'nested', 'optional': False, 'type': ': {\n            nestedProp: T;\n            nestedFunction(): void;\n        }', 'kind': 'property'}}


{'simpleProp': {'name': 'simpleProp',
  'optional': False,
  'type': ': string',
  'kind': 'property'},
 'optionalProp': {'name': 'optionalProp',
  'optional': False,
  'type': ': number',
  'kind': 'property'},
 'functionTypeProp': {'name': 'functionTypeProp',
  'optional': False,
  'type': ': (x: T) => boolean',
  'kind': 'function',
  'parameters': [{'type': ': T', 'optional': False, 'rest': False}]},
 'arrayProp': {'name': 'arrayProp',
  'optional': False,
  'type': ': number[]',
  'kind': 'property'},
 'unionProp': {'name': 'unionProp',
  'optional': False,
  'type': ': string | number',
  'kind': 'property'},
 'nested': {'name': 'nested',
  'optional': False,
  'type': ': {\n            nestedProp: T;\n            nestedFunction(): void;\n        }',
  'kind': 'property'}}

In [ ]:
import cosmograph

cosmograph

import jy

jy.func_name_and_params_pairs

<function jy.js_parse.func_name_and_params_pairs(js_code: str, *, encoding=None)>

# Technique demos

In [ ]:
from i2 import Sig

@Sig('(' + ','.join(['foo', 'bar', 'baz']) + ')')
def foo(*args, **kwargs):
    print(args, kwargs)

Sig(foo)

help(foo)

Help on function foo in module __main__:

foo(foo, bar, baz)



# Parsing the interface out of TS files

## Five files

In [ ]:
from operator import attrgetter, itemgetter  # builtin

# need to pip install
import requests
from dol import Pipe 
from jy.ts_parse import parse_ts_with_oa

from cosmograph.wip.config_properties import my_ts_parser


In [ ]:
import os 
from pathlib import Path
from functools import partial

from lkj import unique_affixes


unique_relative_paths = partial(
    unique_affixes,
    suffix=True,
    ingress=lambda p: Path(p).parts,
    egress=os.path.sep.join,
)

# CHANGE THIS TO THE DIRECTORY WHERE cosmograph IS LOCATED
cosmograph_rootdir = '/Users/thorwhalen/Dropbox/py/proj/c/cosmograph'

config_ts_subdir = 'packages/cosmograph/src/modules/cosmograph/config'
config_ts_dir = os.path.join(cosmograph_rootdir, config_ts_subdir)

# list full paths of dir
ts_paths = list(map(str, Path(config_ts_dir).iterdir()))

ts_paths = dict(zip(unique_relative_paths(ts_paths), ts_paths))

ts_paths['cosmos/config.ts'] = '/Users/thorwhalen/Dropbox/py/proj/c/cosmos/src/config.ts'

ts_paths

# _json_schemas = dict(zip(ts_paths, map(Pipe(lambda fp: Path(fp).read_text(), my_ts_parser), ts_paths.values())))
# list(_json_schemas)
## ['data.ts', 'config.ts', 'labels.ts', 'simulation.ts', 'cosmos/config.ts']

{'data.ts': '/Users/thorwhalen/Dropbox/py/proj/c/cosmograph/packages/cosmograph/src/modules/cosmograph/config/data.ts',
 'config.ts': '/Users/thorwhalen/Dropbox/py/proj/c/cosmograph/packages/cosmograph/src/modules/cosmograph/config/config.ts',
 'labels.ts': '/Users/thorwhalen/Dropbox/py/proj/c/cosmograph/packages/cosmograph/src/modules/cosmograph/config/labels.ts',
 'simulation.ts': '/Users/thorwhalen/Dropbox/py/proj/c/cosmograph/packages/cosmograph/src/modules/cosmograph/config/simulation.ts',
 'cosmos/config.ts': '/Users/thorwhalen/Dropbox/py/proj/c/cosmos/src/config.ts'}

In [ ]:
# import json

# json.dump(_json_schemas, open('ts_json_schemas.json', 'w'), indent=2)

## Nikita's config

In [ ]:
from cosmograph.wip.config_properties import my_ts_parser
from graze import graze

nikitas_belief_url = 'https://raw.githubusercontent.com/cosmograph-org/cosmograph/92bf38fc0674643dcc91246d4352441219a701b0/packages/cosmograph/src/cosmograph/config/config.ts?token=GHSAT0AAAAAACSGWAOWPAK7V3BRRLXVFJ5CZ2IGIKA'

ts_code = graze(nikitas_belief_url).decode()
nikita_json_schema = my_ts_parser(ts_code)
list(nikita_json_schema)



['interfaces']

In [ ]:
t = nikita_json_schema['interfaces']
print(f"{len(t)} interfaces")
print(f"{[x['name'] for x in t]=}\n")
print("First two:")
list(t)[:1]

3 interfaces
[x['name'] for x in t]=['BasicConfig', 'CallbackConfig', 'CosmographConfig']

First two:


[{'name': 'BasicConfig',
  'description': 'Contains basic configuration options for the cosmograph.',
  'properties': [{'name': 'disableSimulation',
    'type': 'boolean | null',
    'description': 'Do not run the simulation, just render the graph. Defaults to true if null and no links are present in the data.',
    'optional': True,
    'default': 'null'},
   {'name': 'rectangularSelectorClassName',
    'type': 'string',
    'description': 'Specifies the CSS class to use for the rectangular selector.',
    'optional': True}]}]

In [ ]:
def edit_nikita_config(nikita_config):
    from functools import partial
    from lkj import get_by_value

    get_by_name = partial(get_by_value, field='name')

    # add the {"left": 7, "top": 4, "right": 7, "bottom": 4} default to labelPadding
    d = get_by_name(nikita_config['interfaces'], 'CosmographConfig')
    d = get_by_name(d['properties'], 'labelPadding')
    d['default'] = {"left": 7, "top": 4, "right": 7, "bottom": 4}

    return nikita_config

nikita_json_schema = edit_nikita_config(nikita_json_schema)


In [ ]:
# from cosmograph.util import data_files
# import json 

# data_files['nikita_config.json'] = json.dumps(nikita_json_schema).encode()

In [ ]:
from cosmograph.util import data_files
import json 
from cosmograph.wip.config_properties import print_interfaces

nikita_config = json.loads(data_files['nikita_config.json'])
print(f"{[x['name'] for x in nikita_config['interfaces']]=}\n")


[x['name'] for x in nikita_config['interfaces']]=['BasicConfig', 'CallbackConfig', 'CosmographConfig']



In [ ]:
from cosmograph.wip.config_properties import print_interfaces

print_interfaces(nikita_config['interfaces'], 'nikita_config')

* **nikita_config**
    * **BasicConfig**: (2)
        disableSimulation, rectangularSelectorClassName
    * **CallbackConfig**: (5)
        onPointsFiltered, onLinksFiltered, onLabelClick, onDataUpdated, onAreaSelected
    * **CosmographConfig**: (7)
        disableSimulation, showTopLabelsLimit, showHoveredPointLabel, staticLabelWeight, dynamicLabelWeight,
        labelMargin, labelPadding


In [ ]:
from cosmograph.wip.config_properties import process_schema_object, json_schema_to_dataframe, dataframes_to_markdown

def config_edits(config):
    t = config['interfaces']['CosmographConfig'] = process_schema_object(config['interfaces'])
    return config


t = dataframes_to_markdown(
    json_schema_to_dataframe(nikita_config),
    # columns=['name', 'description', 'default', 'type'],
)
print(''.join(t))

## interfaces

### BasicConfig

| name                         | type           | description                                                                                                      | optional   | default   |
|:-----------------------------|:---------------|:-----------------------------------------------------------------------------------------------------------------|:-----------|:----------|
| disableSimulation            | boolean | null | Do not run the simulation, just render the graph. Defaults to true if null and no links are present in the data. | True       | null      |
| rectangularSelectorClassName | string         | Specifies the CSS class to use for the rectangular selector.                                                     | True       | nan       |


### CallbackConfig

| name             | type                                                                                                                                                                 

## Study the json_schema a bit

In [ ]:
import json

_json_schemas = json.load(open('ts_json_schemas.json', 'r'))
list(_json_schemas)

['data.ts', 'config.ts', 'labels.ts', 'simulation.ts', 'cosmos/config.ts']

In [ ]:
if all(map(lambda x: list(x) == ['interfaces'], _json_schemas.values())):
    print('all json_schemas have interface and only interface, so removing that field')
    json_schemas = {k: v['interfaces'] for k, v in _json_schemas.items()}


all json_schemas have interface and only interface, so removing that field


In [ ]:
# Make a markdown list for the schema properties
from cosmograph.wip.config_properties import print_interfaces

for k, v in json_schemas.items():    
    print_interfaces(v, k)

* **data.ts**
    * **CosmographPointsConfig**: (12)
        Points, PointId, PointIndex, PointColor, PointColorFn, PointSize, PointSizeFn, PointLabel,
        PointLabelWeight, PointX, PointY, PointIncludeColumns
    * **CosmographLinksConfig**: (14)
        Links, LinkSource, LinkSourceIndex, LinkTarget, LinkTargetIndex, LinkColor, LinkColorFn, LinkWidth,
        LinkWidthFn, LinkArrow, LinkArrowFn, LinkStrength, LinkStrengthFn, LinkIncludeColumns
* **config.ts**
    * **BasicConfig**: (2)
        disableSimulation, rectangularSelectorClassName
    * **CallbackConfig**: (5)
        onPointsFiltered, onLinksFiltered, onLabelClick, onDataUpdated, onAreaSelected
    * **CosmographConfig**: (7)
        disableSimulation, showTopLabelsLimit, showHoveredPointLabel, staticLabelWeight, dynamicLabelWeight,
        labelMargin, labelPadding
* **labels.ts**
    * **LabelsCosmographConfig**: (15)
        showDynamicLabels, showLabelsFor, showTopLabels, showTopLabelsLimit, showTopLabelsBy, pointL

In [ ]:
from i2 import postprocess

@postprocess(dict)
def _property_groups():
    for k, v in json_schemas.items():
        for interface in v:
            yield interface['name'], interface['properties']

property_groups = _property_groups()


property_groups = _property_groups()
list(property_groups)

['CosmographPointsConfig',
 'CosmographLinksConfig',
 'BasicConfig',
 'CallbackConfig',
 'CosmographConfig',
 'LabelsCosmographConfig',
 'SimulationConfig',
 'SimulationEventConfig',
 'GraphEvents',
 'GraphSimulationSettings',
 'GraphConfigInterface']

In [ ]:
# Make a rjsf form for the schema properties
# copy the output here and go to https://rjsf-team.github.io/react-jsonschema-form/
# replacing the "properties" field with the output of this cell

def list_of_dicts_to_dict(d: dict, key_name: str) -> dict:
    return {x[key_name]: {k: x[k] for k in x if k != 'name'} for x in d}

def filter_in_types(d: dict, allowed_types: list) -> dict:
    """Keep only the allowed_types"""
    return {k: v for k, v in d.items() if v['type'] in allowed_types}

t = list_of_dicts_to_dict(property_groups['GraphConfigInterface'], key_name='name')
t = filter_in_types(t, ['string', 'number', 'boolean'])
import json

print(json.dumps(t, indent=2))

{
  "disableSimulation": {
    "type": "boolean",
    "description": "Disable simulation and render graph using predefined point positions. Default is false.",
    "optional": true,
    "default": false
  },
  "spaceSize": {
    "type": "number",
    "description": "Simulation space size with a max of 8192. Default is 4096.",
    "optional": true,
    "default": 4096
  },
  "pointGreyoutOpacity": {
    "type": "number",
    "description": "Opacity of points when greyed-out during selection. Default is 0.1.",
    "optional": true,
    "default": 0.1
  },
  "defaultPointSize": {
    "type": "number",
    "description": "Default size for points when unspecified. Default size is 4.",
    "optional": true,
    "default": 4
  },
  "pointSizeScale": {
    "type": "number",
    "description": "Scale factor for point size. Default is 1.",
    "optional": true,
    "default": 1
  },
  "hoveredPointCursor": {
    "type": "string",
    "description": "Cursor style when hovering over a point. Defau

In [ ]:
# to more:
import ju.rjsf


<module 'ju.rjsf' from '/Users/thorwhalen/Dropbox/py/proj/i/ju/ju/rjsf.py'>

In [ ]:
# from pprint import pprint

# pprint(property_groups)

In [ ]:
from cosmograph.wip.config_properties import process_schema_object, json_schema_to_dataframe, dataframes_to_markdown

t = ''.join(dataframes_to_markdown(
    json_schema_to_dataframe(json_schemas),
    columns=['name', 'description', 'default', 'type'],
))
print(t)

## data.ts

### CosmographPointsConfig

| name                | description                                                                                                                       | type                                                  |
|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------|
| Points              | Input data for the points. Accepts `File | string | Table | Uint8Array | ArrayBuffer | Record<string, unknown>[]`                 | CosmographInputData                                   |
| PointId             | Unique identifier column for each point. Required for mapping links to points correctly.                                          | string                                                |
| PointIndex          | Numeric index column for each point. Used for efficient lookups and should be a sequenti

In [ ]:
df = json_schema_to_dataframe(json_schemas)
list(df)

['data.ts', 'config.ts', 'labels.ts', 'simulation.ts', 'cosmos/config.ts']

,name,type,description,optional,default
0,simulationDecay,GraphSimulationSettings['decay'],Decay coefficient. Use bigger values if you wa...,True,1000
1,simulationGravity,GraphSimulationSettings['gravity'],Gravity force coefficient.,True,0
2,simulationCenter,GraphSimulationSettings['center'],Centering to center mass force coefficient.,True,0
3,simulationRepulsion,GraphSimulationSettings['repulsion'],Repulsion force coefficient.,True,0.1
4,simulationRepulsionTheta,GraphSimulationSettings['repulsionTheta'],Decreases / increases the detalization of the ...,True,1.7
5,simulationRepulsionQuadtreeLevels,GraphSimulationSettings['repulsionQuadtreeLeve...,Barnes–Hut approximation depth. Can only be us...,True,12
6,simulationLinkSpring,GraphSimulationSettings['linkSpring'],Link spring force coefficient.,True,1
7,simulationLinkDistance,GraphSimulationSettings['linkDistance'],Minimum link distance.,True,2
8,simulationLinkDistRandomVariationRange,GraphSimulationSettings['linkDistRandomVariati...,Range of random link distance values.,True,"[1, 1.2]"
9,simulationRepulsionFromMouse,GraphSimulationSettings['repulsionFromMouse'],Repulsion coefficient from mouse position. The...,True,2


In [ ]:
t = property_groups['GraphSimulationSettings']
t[0]

{'name': 'decay',
 'type': 'number',
 'description': 'Decay coefficient controlling simulation cooldown. Default is 5000.',
 'optional': True,
 'default': 5000}

# Orphan utils developped on the way

See [github_raw_utils.py gist](https://gist.github.com/thorwhalen/92fa45e8f36f8caf155ea3012ad61629): `ensure_raw_github_url`, `get_url_text`, `ensure_text`, `might_be_url`.
